In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d1 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies_own = {
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    #"red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    #"red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
policies_enem = {
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
# policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 1200*25,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
config_enem = ppo.DEFAULT_CONFIG.copy()
config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

adversary = ppo.PPOTrainer(config=config_enem,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    #reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    #reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    reload_weights(policy_id="blue_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="blue_1",trainer=trainer,set_policy_id="blue_1")
    #save_weights("red_0",trainer)
    #save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-04-07 20:59:42,993	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-04-07 20:59:42,995	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi

2022-04-07 20:59:57,223	INFO trainable.py:130 -- Trainable.setup took 14.232 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-07 20:59:57,225	WARNING util.py:55 -- Install gputil for GPU system monitoring.


2022-04-07 21:00:10,475	INFO trainable.py:130 -- Trainable.setup took 13.204 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-07 21:00:10,477	WARNING util.py:55 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.self_play = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)

                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])


                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()

                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue_"+str(i),steps,env_blue_pos_mod,hist_dir)
                    save_hists("red_"+str(i),steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm_"+str(i),steps,env_mrm_pos_mod,hist_dir)
                    save_weights("blue_0",trainer)
                    save_weights("blue_1",trainer)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
281 blue_1 Shoot at red_0 launch distance : 58575.91101654219 True True
293 blue_1 Shoot at red_0 launch distance : 53394.695397068186 True True
319 red_1 Shoot at blue_1
330 red_1 Shoot at blue_1
390 blue_1: Destroyed
394 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 394 -1.1010072150031247 -3.135024874057661
blue_1 False False 394 0.3990002183302086 103.02046686957709
-------------------------- Scene: 0 --------------------------
280 blue_1 Shoot at red_0 launch distance : 59218.49620287246 True True
291 blue_0 Shoot at red_0 launch distance : 57962.42759223243 True True
302 blue_0 Shoot at red_0 launch distance : 52543.82725161311 True True
325 blue_1 DOWN
340 red_1 Shoot at blue_0
351 red_1 Shoot at blue_0
400 blue_0 Splash :red_0
4

271 blue_1 Shoot at red_1 launch distance : 59435.9751389611 True True
285 blue_1 Shoot at red_1 launch distance : 52576.75874984622 True True
320 red_0 Shoot at blue_1
331 red_0 Shoot at blue_1
385 blue_1: Destroyed
386 blue_1 Splash :red_1
Same tgt shoot
Same tgt shoot
639 blue_0 Shoot at red_0 launch distance : 59985.89867416183 True True
Same tgt shoot
Same tgt shoot
650 blue_0 Shoot at red_0 launch distance : 55143.25841973209 True True
758 blue_0 Splash :red_0
WIN
blue_0 False True 850 11.410766139215687 128.79668815214444
blue_1 False True 850 11.310764705882352 132.8828479408795
-------------------------- Scene: 0 --------------------------
255 blue_1 Shoot at red_1 launch distance : 59155.937303095285 True True
268 blue_1 Shoot at red_1 launch distance : 53453.25776522306 True True
323 red_0 Shoot at blue_1
334 red_0 Shoot at blue_1
370 blue_1 Splash :red_1
393 blue_1: Destroyed
Same tgt shoot
Same tgt shoot
727 blue_0 Shoot at red_0 launch distance : 59578.483141560064 True T

726 blue_0 Splash :red_0
753 blue_0 Splash :red_1
WIN
blue_0 False True 753 25.28142533775647 118.41717438465749
blue_1 False True 753 11.994165375073868 25.180463436573596
-------------------------- Scene: 0 --------------------------
171 blue_1 DOWN
541 blue_0 Shoot at red_1 launch distance : 59221.7136014687 True True
553 blue_0 Shoot at red_1 launch distance : 53832.22101283202 True True
595 red_0 Shoot at blue_0
606 red_0 Shoot at blue_0
657 blue_0 Splash :red_1
662 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 662 0.8604767233214271 119.79157783900365
blue_1 False False 662 0.8989996899880938 -0.40520746263996454
-------------------------- Scene: 0 --------------------------
285 blue_1 Shoot at red_0 launch distance : 59046.930727253006 True True
296 blue_1 Shoot at red_0 launch distance : 53590.00028830679 True True
333 red_1 Shoot at blue_1
344 red_1 Shoot at blue_1
398 blue_1 Splash :red_0
406 blue_1: Destroyed
Same tgt shoot
Same tgt shoot
705 blue_0 Shoot at red_1 lau

361 blue_1 Shoot at red_1 launch distance : 59941.170291904426 True True
378 blue_1 Shoot at red_1 launch distance : 52743.25735536446 True True
421 red_0 Shoot at blue_1
432 red_0 Shoot at blue_1
481 blue_1 Splash :red_1
488 blue_1: Destroyed
Same tgt shoot
Same tgt shoot
717 blue_0 Shoot at red_0 launch distance : 34142.181515898934 True True
Same tgt shoot
Same tgt shoot
728 blue_0 Shoot at red_0 launch distance : 29689.033107771313 True True
776 blue_0 Splash :red_0
WIN
blue_0 False True 928 11.292101614942528 80.61789392009727
blue_1 False True 928 11.19210344827586 137.19248192584755
-------------------------- Scene: 0 --------------------------


2022-04-07 22:01:28,284	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


...

TIME LIMIT LOSE
blue_0 False False 1200 0.8930004573767767 -23.029217568762686
blue_1 False False 1200 0.8990004573767767 121.12483303837263
-------------------------- Scene: 0 --------------------------
287 blue_1 Shoot at red_1 launch distance : 53340.16310356028 True True
298 blue_1 Shoot at red_1 launch distance : 48094.6149567508 True True
329 red_0 Shoot at blue_1
340 red_0 Shoot at blue_1
384 blue_1 Splash :red_1
398 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.9930008269792536 -18.824042589474626
blue_1 False False 1200 0.8990008269792537 114.06779565929396
-------------------------- Scene: 0 --------------------------
294 blue_1 Shoot at red_1 launch distance : 52451.55549409128 True True
338 red_0 Shoot at blue_1
349 red_0 Shoot at blue_1
383 blue_1 Shoot at red_1 launch distance : 51355.943801908244 True True
398 blue_1 Splash :red_1
418 blue_1: Destroyed
Same tgt shoot
Same tgt shoot
895 blue_0 Shoot at red_0 launch distance : 53249.78759232562 True True
1014

435 blue_0 Splash :red_0
439 blue_0 Splash :red_1
WIN
blue_0 False True 439 28.7008146327108 159.76894052684264
blue_1 False True 439 12.733850145467063 9.6225768269552
-------------------------- Scene: 0 --------------------------
299 blue_1 Shoot at red_0 launch distance : 59966.585673240574 True True
310 blue_1 Shoot at red_0 launch distance : 54912.80792508323 True True
353 red_1 Shoot at blue_1
364 red_1 Shoot at blue_1
415 blue_1 Splash :red_0
422 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.9939999849324367 -13.847125122718465
blue_1 False False 1200 0.8989999849324366 103.28285409037318
-------------------------- Scene: 0 --------------------------
364 blue_1 Shoot at red_0 launch distance : 57857.01039414631 True True
375 blue_1 Shoot at red_0 launch distance : 52142.9364656061 True True
406 red_1 Shoot at blue_1
417 red_1 Shoot at blue_1
467 blue_1 Splash :red_0
477 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.9940088049147439 -18.63950892804392


2022-04-08 07:16:33,693	INFO trainable.py:496 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000010\checkpoint-10
2022-04-08 07:16:33,693	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 10, '_timesteps_total': 304663, '_time_total': 36127.51764059067, '_episodes_total': 350}


agent_timesteps_total: 609326
custom_metrics: {}
date: 2022-04-08_07-16-33
done: false
episode_len_mean: 780.95
episode_media: {}
episode_reward_max: 307.9013353301605
episode_reward_mean: 145.7680641985548
episode_reward_min: -82.58325569561018
episodes_this_iter: 35
episodes_total: 350
experiment_id: 19b7e384e0e5486f9550b1be53a9ccca
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 7.110646724700928
        entropy_coeff: 0.0
        kl: 0.19716201722621918
        model: {}
        policy_loss: 0.02923949435353279
        total_loss: 149.4951934814453
        vf_explained_var: -0.16619282960891724
        vf_loss: 149.426513671875
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.612335205078125
        entropy_coeff: 0.0
        kl: 0.13707058131694794
        model: {}


282 blue_1 Shoot at red_0 launch distance : 59569.80751283551 True True
296 blue_1 Shoot at red_0 launch distance : 52816.61478370221 True True
309 blue_0 DOWN
331 red_1 Shoot at blue_1
342 red_1 Shoot at blue_1
396 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 396 -0.10100050248144143 -1.3446577496502798
blue_1 False False 396 0.3586490308518919 103.54313527426281
-------------------------- Scene: 0 --------------------------
307 blue_0 Shoot at red_1 launch distance : 58357.99134901283 True True
329 blue_0 Shoot at red_1 launch distance : 49770.054414735685 True True
346 red_0 Shoot at blue_0
357 red_0 Shoot at blue_0
408 blue_0: Destroyed
421 blue_0 Splash :red_1
562 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 562 0.8990002597629094 81.8571474718084
blue_1 False False 562 -0.10699974023709058 -6.595233654174366
-------------------------- Scene: 0 --------------------------
286 blue_1 Shoot at red_0 launch distance : 58825.65501822114 True True
Same tgt shoot
317 blue_1 Sho